## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [1]:
import numpy as np
import random

random.seed()

In [2]:
def initialize_individual(num_features):
    return np.random.permutation(num_features)

In [3]:
def initialize_population(num_individuals, num_features):
    return np.array([initialize_individual(num_features) for _ in range(num_individuals)])

In [12]:
def fitness(weights, individual):
    used_first_diag = {}
    used_secondary_diag = {}
    answer_sum = 0
    for i in range(individual.shape[0]):
        secondary_diag = i + individual[i]
        first_diag = i - individual[i]
        
        if first_diag in used_first_diag:
            return 0
    
        if secondary_diag in used_secondary_diag:
            return 0
        
        used_first_diag[first_diag] = 1
        used_secondary_diag[secondary_diag] = 1
        answer_sum += weights[i][individual[i]] 
    return answer_sum

In [5]:
def selection(weights, individuals):
    fitnesses = np.array([fitness(weights, individual) for individual in individuals]) + 1
    probabilities = fitnesses / fitnesses.max(axis = 0)
    selected = []
    for i in range(individuals.shape[0]):
        if random.random() < probabilities[i]:
            selected.append(individuals[i])
    return np.array(selected)

In [6]:
def crossover(individuals):
    indexes = np.random.choice(individuals.shape[0], size = 2, replace=False)
    child = np.zeros(individuals[indexes[0]].shape[0]).astype(int)
    for i in range(individuals[indexes[0]].shape[0]):
        child[i] = individuals[indexes[0]][individuals[indexes[1]][i]]
    return child

In [7]:
def mutation(individual, mutation_rate):
    if random.random() > mutation_rate:
        return individual
    indexes = np.random.choice(individual.shape[0], size = 2, replace=False)
    individual[indexes[0]], individual[indexes[1]] = individual[indexes[1]], individual[indexes[0]]
    return individual

In [8]:
def get_new_generation(weights, individuals, mutation_rate):
    selected = selection(weights, individuals)
    new_generation = [individual for individual in selected]
    for _ in range(selected.shape[0], individuals.shape[0]):
        new_generation.append(np.array(mutation(crossover(selected), mutation_rate)))
    return np.array(new_generation)

In [9]:
def run_simulation(generations, number_individuals, weights, mutation_rate):
    individuals = initialize_population(number_individuals, weights.shape[0])
    for i in range(generations):
        if i % 100 == 0:
            print(f"Done {i} generations")
        individuals = get_new_generation(weights, individuals, mutation_rate)
    fitnesses = np.array([fitness(weights, individual) for individual in individuals])
    return individuals[np.argmax(fitnesses)]

In [13]:
weights = np.random.rand(8, 8)
best = run_simulation(1000, 1000, weights, 0.1)
print(f"Weights are {weights}")
print(f"Found {best} found with fitness {fitness(weights, best)}")

Done 0 generations
Done 100 generations
Done 200 generations
Done 300 generations
Done 400 generations
Done 500 generations
Done 600 generations
Done 700 generations
Done 800 generations
Done 900 generations
Weights are [[0.18241631 0.20515543 0.52252729 0.88077913 0.23583217 0.98843977
  0.72767396 0.41251971]
 [0.68050388 0.93632896 0.3910408  0.80860707 0.11311232 0.1866865
  0.81377616 0.86851949]
 [0.80904211 0.43536879 0.03982579 0.96746336 0.71864374 0.86881255
  0.98122102 0.23475136]
 [0.23490856 0.06186793 0.98480608 0.83685528 0.77423255 0.29164603
  0.73053667 0.35268587]
 [0.07321055 0.64836406 0.0939527  0.19666217 0.41988896 0.15528018
  0.48133175 0.25983403]
 [0.28029254 0.48199156 0.64413658 0.76568752 0.32269025 0.61501581
  0.79750337 0.96442485]
 [0.43034529 0.71174235 0.54684658 0.29805377 0.38721234 0.68633411
  0.29961663 0.81992023]
 [0.95558992 0.53200389 0.44388923 0.77075081 0.46534182 0.84123844
  0.65885701 0.37226274]]
Found [3 1 6 2 5 7 4 0] found with f